In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [ ]:
from math import radians, cos, sin, asin, sqrt

def dist(a, b):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    x1, y1, x2, y2 = map(radians, [a[0], a[1], b[0], b[1]])
    dx = x2 - x1
    dy = y2 - y1
    a = sin(dy/2)**2 + cos(y1) * cos(y2) * sin(dx/2)**2
    c = 2 * asin(sqrt(a))
    # Earth`s radius in kilometers
    km = 6372.795 * c
    return km


In [ ]:
meteo_path = "G:/1_Data1/90_BaikalJun2022Sea/Meteo/"
origin_path = "G:/1_Data1/90_BaikalJun2022Sea/jun2022sea.dat"

In [ ]:
df = pd.read_csv(origin_path,
                        header=0,
                        skiprows=1,
                        sep="\t",
                        na_values='--',
                        decimal=',',
                        parse_dates=['DateTime'],
                        dayfirst=True,
                        )
#origin_df.info()
df.sort_values(by='DateTime', inplace=True)

In [ ]:
print(df.columns)

In [ ]:
df.info()

In [ ]:
df

In [ ]:
def get_ang(x: float, y: float) -> any:
    length = get_len(x, y)
    if length == 0:
        return None
    ang = np.arctan2(x, -y) * 180 / np.pi
    return (ang + 270) % 360

lat_rad1 = df['Latitude'] * np.pi / 180
long_rad1 = df['Longitude'] * np.pi / 180
lat_rad2 = lat_rad1.shift(1)
long_rad2 = long_rad1.shift(1)

dTime = (df['DateTime'] - df['DateTime'].shift(1)).astype('int64')/10**9

cl1 = np.cos(lat_rad1)
cl2 = np.cos(lat_rad2)
sl1 = np.sin(lat_rad1)
sl2 = np.sin(lat_rad2)
delta = long_rad2 - long_rad1
cdelta = np.cos(delta)
sdelta = np.sin(delta)

y = np.sqrt(np.power(cl2*sdelta, 2) + np.power(cl1*sl2-sl1*cl2*cdelta, 2))
x = sl1*sl2+cl1*cl2*cdelta
ad = math.atan2(y,x)
df['dist1'] = ad * 6372800

x = (cl1*sl2) - (sl1*cl2*cdelta)
y = sdelta*cl2
z = math.degrees(math.atan(-y/x))
if (x < 0):
    z = z+180.

z2 = (z+180.) % 360. - 180.
z2 = - math.radians(z2)
anglerad2 = z2 - ((2*math.pi)*math.floor((z2/(2*math.pi))) )
angledeg = (anglerad2*180.)/math.pi

a = np.sin(dLat/2)**2 + np.cos(long_rad) * np.cos(long_shifted) * np.sin(dLong)**2
df['dist'] = (2 * np.arcsin(np.sqrt(a)) * 6372.8 * 1000)
df['speed'] = df['dist'] / dTime

dist_east = np.cos(lat_rad) * dLong
dist_nord = dLat
df['course'] = np.arctan2(dLong, dLat) * 180 / np.pi

In [ ]:
lat1 = llat1*math.pi/180.
lat2 = llat2*math.pi/180.
long1 = llong1*math.pi/180.
long2 = llong2*math.pi/180.

 #косинусы и синусы широт и разницы долгот
cl1 = math.cos(lat1)
cl2 = math.cos(lat2)
sl1 = math.sin(lat1)
sl2 = math.sin(lat2)
delta = long2 - long1
cdelta = math.cos(delta)
sdelta = math.sin(delta)
#вычисления длины большого круга
y = math.sqrt(math.pow(cl2*sdelta,2)+math.pow(cl1*sl2-sl1*cl2*cdelta,2))
x = sl1*sl2+cl1*cl2*cdelta
ad = math.atan2(y,x)
dist = ad*rad
#вычисление начального азимута
x = (cl1*sl2) - (sl1*cl2*cdelta)
y = sdelta*cl2
z = math.degrees(math.atan(-y/x))
if (x < 0):
    z = z+180.

z2 = (z+180.) % 360. - 180.
z2 = - math.radians(z2)
anglerad2 = z2 - ((2*math.pi)*math.floor((z2/(2*math.pi))) )
angledeg = (anglerad2*180.)/math.pi


In [ ]:
df.loc[(df['DateTime'] > pd.to_datetime('2022-06-03 12:00')) & (df['DateTime'] < pd.to_datetime('2022-06-03 13:00'))]

In [ ]:
cols = ['Longitude', 'Latitude', 'TempAir', 'PressAir',
        'TempEqu1', 'dateSec', 'vCO2Air', 'vCH4Air', 'pCO2Air', 'pCH4Air', 'cCO2Wtr', 'pCO2Wtr', 'cCH4Wtr', 'pCH4Wtr']
cols_dict = {}
for item in cols:
    cols_dict[item] = 'mean'

In [ ]:
cols_dict

In [ ]:
origin_df_minutly = origin_df.resample('1T', on='DateTime').agg(cols_dict).ffill()
origin_df_minutly

In [ ]:
meteo_files_list = glob.glob(meteo_path+"*.txt")
dfl = []
for f in meteo_files_list:
    df = pd.read_csv(f,
                     encoding_errors="ignore",
                     header=None,
                     skiprows=1,
                     sep=" ",
                     names=['date', 'time', 't_air', 'w_hor', 'w_dir', 'wx', 'wy', 'wz', 'press', 'rh']
                    )
    dfl.append(df)

meteo_df = pd.concat(dfl)
meteo_df['DateTime'] = pd.to_datetime(meteo_df['date']+' '+meteo_df['time'], dayfirst=True, cache=True)
meteo_df.sort_values(by='DateTime', inplace=True)
meteo_df


In [ ]:
np.rad2deg(-np.arctan2(-meteo_df['wy'], -meteo_df['wx']))

In [ ]:
print(df.columns)
cols = ['t_air', 'w_hor', 'w_dir', 'press', 'rh']
cols_dict = {}
for item in cols:
    cols_dict[item] = 'mean'

In [ ]:
meteo_df_minutly = meteo_df.resample("1T", on="DateTime", ).agg(cols_dict).ffill()
meteo_df_minutly

In [ ]:
df = pd.merge(origin_df_minutly, meteo_df_minutly, how='outer', on='DateTime')
df.sort_values(by='DateTime', inplace=True)
df

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,5)
ax.set_ylim(0, 80)
ax.set_xlim(pd.to_datetime('2022/06/01'),pd.to_datetime('2022/06/15'))
ax.grid()
ax.plot(df.index, 'speed', 'g-', data=df)

In [ ]:
df.to_csv('G:/1_Data1/90_BaikalJun2022Sea/jun2022sea_pandas.out', sep='\t')

In [ ]:
plt.isinteractive()